<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip 

--2022-09-26 18:42:46--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  26.9MB/s    in 0.2s    

2022-09-26 18:42:46 (26.9 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-09-26 18:42:46--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [121]:
import pandas as pd
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model


# Data

In [122]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
print(data.shape)
print(data.user_id.nunique())
print(data.movie_id.nunique())
data.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(10000, 10)
46
2175


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation,F,1,10,48067


# Feature Engineering

In [123]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [124]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

In [125]:
feature_max_idx

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [126]:
data.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18


In [127]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')
user_profile.shape, item_profile.shape

((46, 4), (2175, 1))

In [128]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list.head()

user_id
1    [644, 385, 483, 1881, 1306, 646, 731, 1570, 35...
2    [644, 1735, 1283, 1077, 656, 1121, 692, 773, 1...
3    [644, 1570, 487, 647, 357, 693, 909, 670, 72, ...
4    [644, 1881, 1735, 1283, 320, 1545, 152, 1121, ...
5    [644, 1306, 359, 1283, 405, 715, 320, 604, 154...
Name: movie_id, dtype: object

In [129]:
train_set, test_set = gen_data_set(data, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 46/46 [00:00<00:00, 288.17it/s]


7 8


In [130]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [131]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((19816,), (19816,), (46,), (46,))

In [132]:
train_model_input

{'user_id': array([34, 42,  2, ..., 10,  6, 43]),
 'movie_id': array([1399, 1114, 1918, ...,  997, 1594,  338]),
 'hist_movie_id': array([[ 597,  355,  708, ...,   38,  653,  320],
        [ 571,  155, 1245, ..., 1358, 1230, 2072],
        [ 357, 1726,  656, ...,    0,    0,    0],
        ...,
        [ 678, 1860, 1956, ..., 1434,  608,  629],
        [ 220, 1196, 1150, ..., 1902,  887, 1248],
        [ 305,  220, 1267, ...,   14,  662, 1650]], dtype=int32),
 'hist_genres': array([[ 8,  2,  8, ...,  6,  2,  8],
        [ 8,  8,  5, ...,  1,  1,  1],
        [ 8,  8,  8, ...,  0,  0,  0],
        ...,
        [ 5,  6,  5, ..., 12,  8,  5],
        [ 3,  3,  3, ...,  1,  1,  1],
        [ 8,  3,  8, ...,  8,  6,  8]], dtype=int32),
 'hist_len': array([50, 50, 16, ..., 50, 50, 50]),
 'genres': array([ 5,  8,  8, ...,  1,  8, 18]),
 'gender': array([2, 2, 2, ..., 2, 1, 2]),
 'age': array([4, 2, 7, ..., 5, 2, 3]),
 'occupation': array([ 8,  5, 13, ...,  4,  4,  6]),
 'zip': array([29, 37, 

# Specify Model Features Input

In [133]:
# from tensorflow.python.keras.initializers import RandomNormal, Zeros
# from collections import namedtuple, OrderedDict
# class SparseFeat(namedtuple('SparseFeat',
#                             ['name', 'vocabulary_size', 'embedding_dim', 'use_hash', 'vocabulary_path', 'dtype', 'embeddings_initializer',
#                              'embedding_name',
#                              'trainable'])):
#     __slots__ = ()

#     def __new__(cls, name, vocabulary_size, embedding_dim=4, use_hash=False, vocabulary_path=None, dtype="int32", embeddings_initializer=None,
#                 embedding_name=None, trainable=True):

#         if embedding_dim == "auto":
#             embedding_dim = 6 * int(pow(vocabulary_size, 0.25))
#         if embeddings_initializer is None:
#             embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)

#         if embedding_name is None:
#             embedding_name = name

#         return super(SparseFeat, cls).__new__(cls, name, vocabulary_size, embedding_dim, use_hash, vocabulary_path, dtype,
#                                               embeddings_initializer,
#                                               embedding_name, trainable)

#     def __hash__(self):
#         return self.name.__hash__()

In [134]:
train_model_input

{'user_id': array([34, 42,  2, ..., 10,  6, 43]),
 'movie_id': array([1399, 1114, 1918, ...,  997, 1594,  338]),
 'hist_movie_id': array([[ 597,  355,  708, ...,   38,  653,  320],
        [ 571,  155, 1245, ..., 1358, 1230, 2072],
        [ 357, 1726,  656, ...,    0,    0,    0],
        ...,
        [ 678, 1860, 1956, ..., 1434,  608,  629],
        [ 220, 1196, 1150, ..., 1902,  887, 1248],
        [ 305,  220, 1267, ...,   14,  662, 1650]], dtype=int32),
 'hist_genres': array([[ 8,  2,  8, ...,  6,  2,  8],
        [ 8,  8,  5, ...,  1,  1,  1],
        [ 8,  8,  8, ...,  0,  0,  0],
        ...,
        [ 5,  6,  5, ..., 12,  8,  5],
        [ 3,  3,  3, ...,  1,  1,  1],
        [ 8,  3,  8, ...,  8,  6,  8]], dtype=int32),
 'hist_len': array([50, 50, 16, ..., 50, 50, 50]),
 'genres': array([ 5,  8,  8, ...,  1,  8, 18]),
 'gender': array([2, 2, 2, ..., 2, 1, 2]),
 'age': array([4, 2, 7, ..., 5, 2, 3]),
 'occupation': array([ 8,  5, 13, ...,  4,  4,  6]),
 'zip': array([29, 37, 

In [135]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['movie_id'])]
len(train_counter), type(train_counter), len(item_count)

(2175, collections.Counter, 2176)

# Model Training

In [136]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [137]:
# item features
item_feature_columns = ['movie_id']
# User features
user_feature_columns = ['user_id',
                        "gender",
                        "age",
                        "occupation",
                        "zip",
                        'hist_movie_id',
                        'hist_genres'
                        ]


In [138]:
feature_max_idx

{'movie_id': 2176,
 'user_id': 47,
 'gender': 3,
 'age': 8,
 'occupation': 18,
 'zip': 47,
 'genres': 19}

In [139]:
user_sparse_feature_columns = user_feature_columns[0:5]
user_sparse_feature_columns

['user_id', 'gender', 'age', 'occupation', 'zip']

In [140]:
user_seq_sparse_feature_columns = user_feature_columns[5:]
user_seq_sparse_feature_columns 

['hist_movie_id', 'hist_genres']

In [141]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

In [142]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + '_emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=True)
   emb.trainable = True
   sparse_embedding[name] = emb

In [143]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + '_emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [144]:
sparse_embedding

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd459eb6350>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd459eb6c90>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd45a0a4250>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd45a0a45d0>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd45e957ed0>,
 'hist_movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd459eb6550>,
 'hist_genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd45e957ad0>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7fd459eab150>}

### Create Model Inputs

In [145]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )

In [146]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'gender_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'age_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'occupation_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'zip_1:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_movie_id_1:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_genres_1:0' shape=(None, 50) dtype=int32>]

In [147]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [148]:
item_inputs_list

[<tf.Tensor 'movie_id_1:0' shape=(None, 1) dtype=int32>]

### Combine Input (User only)

In [149]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [150]:
user_dnn_input = Flatten()(Concatenate(axis=1)(user_inputs_list))

In [151]:
user_dnn_input.shape[-1]

105

## Model Specification

In [152]:
from tensorflow.keras import layers

In [154]:
# youtubeDNN
YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)
# (user_feature_columns, item_feature_columns,
#                user_dnn_hidden_units=(64, 32),
#                dnn_activation='relu', dnn_use_bn=False,
#                l2_reg_dnn=0, l2_reg_embedding=1e-6, dnn_dropout=0, output_activation='linear', temperature=0.05,
#                sampler_config=None, seed=1024)

# youtube DNN
user_dnn_out = DNN(user_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                       dnn_use_bn, output_activation=output_activation, seed=seed)(user_dnn_input)
# DNN
def __init__(self, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False, output_activation=None,
                 seed=1024, **kwargs):

SyntaxError: ignored

In [155]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [156]:
# build model
from tensorflow.python.keras.layers import Layer, Dropout, Activation
from tensorflow.python.ops.init_ops_v2 import Zeros, glorot_normal
from tensorflow.keras.layers import BatchNormalization
class DNN(Layer):
  def __init__(self, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_batch_norm=False, output_activation=None,
                 seed=1024, **kwargs):
        self.hidden_units = hidden_units
        self.activation = activation
        self.l2_reg = l2_reg
        self.dropout_rate = dropout_rate
        self.use_bn = use_batch_norm
        self.output_activation = output_activation
        self.seed = seed

        super(DNN, self).__init__(**kwargs)

  def build(self, input_shape):
        input_size = input_shape[-1]
        hidden_units = [int(input_size)] + list(self.hidden_units)

        # specifiy weights w
        self.weights = [self.add_weight(name='weight' + str(i),
                                        shape=(
                                            hidden_units[i], hidden_units[i + 1]),
                                        initializer=glorot_normal(
                                            seed=self.seed),
                                        regularizer=l2(self.l2_reg),
                                        trainable=True) for i in range(len(self.hidden_units))]
        # specify bias b
        self.bias = [self.add_weight(name='bias' + str(i),
                                     shape=(self.hidden_units[i],),
                                     initializer=Zeros(),
                                     trainable=True) for i in range(len(self.hidden_units))]

        # if self.use_batch_norm:
        #     self.bn_layers = [BatchNormalization() for _ in range(len(self.hidden_units))]

        # specify dropout
        self.dropout_layers = [Dropout(self.dropout_rate, seed=self.seed + i) for i in
                               range(len(self.hidden_units))]
        
        # specify activation                               
        self.activation_layers = [Activation(self.activation) for _ in range(len(self.hidden_units))]

        # specify output activation
        if self.output_activation:
            self.activation_layers[-1] = Activation(self.output_activation)

        super(DNN, self).build(input_shape) 

  def call(self, inputs, training=None, **kwargs):
        deep_input = inputs

        for i in range(len(self.hidden_units)):
            fc = tf.nn.bias_add(tf.tensordot(
                deep_input, self.weights[i], axes=(-1, 0)), self.bias[i])
            # if self.use_batch_norm:
            #     fc = self.bn_layers[i](fc, training=training)
            fc = self.activation_layers[i](fc, training=training)
            # try:
            #    fc = self.activation_layers[i](fc, training=training)
            # except TypeError as e:  # TypeError: call() got an unexpected keyword argument 'training'
            #    print("make sure the activation function use training flag properly", e)
            #    fc = self.activation_layers[i](fc)
            fc = self.dropout_layers[i](fc, training=training)
            deep_input = fc
            return deep_input
  
  # helper to peek configuration
  def compute_output_shape(self, input_shape):
        if len(self.hidden_units) > 0:
            shape = input_shape[:-1] + (self.hidden_units[-1],)
        else:
            shape = input_shape

        return tuple(shape)

  def get_config(self, ):
        config = {'activation': self.activation, 'hidden_units': self.hidden_units,
                  'l2_reg': self.l2_reg, 'use_bn': self.use_bn, 'dropout_rate': self.dropout_rate,
                  'output_activation': self.output_activation, 'seed': self.seed}
        base_config = super(DNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [157]:
a = DNN(hidden_units = user_dnn_hidden_units, 
        activation='relu', 
        l2_reg=0, 
        dropout_rate=0, 
        use_batch_norm=False, 
        output_activation='linear',
        seed=1024
        )

# DNN(user_dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
#                        dnn_use_bn, output_activation=output_activation, seed=seed)(user_dnn_input)

In [172]:
user_dnn_input.shape[:-1]# + (user_dnn_hidden_units[-1],)

TensorShape([None])

In [ ]:
shape = input_shape[:-1] + (user_dnn_hidden_units[-1],)

In [173]:
a.get_config()

{'name': 'dnn_9',
 'trainable': True,
 'dtype': 'float32',
 'activation': 'relu',
 'hidden_units': (128, 64, 32),
 'l2_reg': 0,
 'use_bn': False,
 'dropout_rate': 0,
 'output_activation': 'linear',
 'seed': 1024}

In [112]:
a.dropout_layers

ListWrapper([<tensorflow.python.keras.layers.core.Dropout object at 0x7fd45a237150>, <tensorflow.python.keras.layers.core.Dropout object at 0x7fd45a17b850>, <tensorflow.python.keras.layers.core.Dropout object at 0x7fd45a168950>])

In [101]:
b

[105, 128, 64, 32]

In [90]:
user_dnn_input.shape

TensorShape([None, 105])

In [91]:
[int(user_dnn_input.shape[-1])] + list(user_dnn_hidden_units)

[105, 128, 64, 32]

In [94]:
b = [int(user_dnn_input.shape[-1])] + list(user_dnn_hidden_units)
b

[105, 128, 64, 32]

In [99]:
 [b[i]+1 for i in range(len(b))]

[106, 129, 65, 33]

In [ ]:
[Layers.add_weight(name='kernel' + str(i),
                                        shape=(
                                            hidden_units[i], hidden_units[i + 1]),
                                        initializer=glorot_normal(
                                            seed=self.seed),
                                        regularizer=l2(self.l2_reg),
                                        trainable=True) for i in range(len(self.hidden_units))]

In [59]:
Layer(  )

In [60]:
a

In [47]:
user_dnn_hidden_units

(128, 64, 32)